In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [3]:
!pip install tensorflow-hub #텐서플로우 허브로부터 사전훈련된모델 이용가능

In [4]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [5]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

from keras.utils import np_utils

import warnings 
warnings.filterwarnings(action='ignore')

In [6]:
#경로 설정
import os
os.chdir('/content/drive/MyDrive/Project/[sai]perfume')

# 데이터 준비
- 테이블 합치기
- 입력형태 맞추기

In [7]:
data = pd.read_csv("data/preprocessed.csv",encoding="utf-8")
label= pd.read_csv("data/labeled_data.csv")

In [8]:
data.columns

Index(['Unnamed: 0', 'gender', 'name', 'accords', 'review', 'tokenized',
       'only_english', 'longer_than_2_A', 'stopwords_removed', 'lemmatizated'],
      dtype='object')

In [9]:
data = data.drop("Unnamed: 0", axis =1)
data.head()

,gender,name,accords,review,tokenized,only_english,longer_than_2_A,stopwords_removed,lemmatizated
0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'sample', 'this', 'today', 'and', 'yea...","['got', 'sample', 'today', 'year', 'old', 'dau...","['get', 'sample', 'today', 'year', 'old', 'dau..."
1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\r\nI have ...,"['first', 'impressions.', 'test', 'on', 'blott...","['first', 'impressions', 'test', 'on', 'blotte...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impression', 'test', 'blotter', 'he..."
2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...,"['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'best', 'friend...","['perfume', 'reminds', 'best', 'friend', 'actu...","['perfume', 'reminds', 'best', 'friend', 'actu..."
3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...,"['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'feet', 'falling', 'fa...","['imagine', 'trip', 'foot', 'fall', 'face', 'f..."
4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\r\nLove the scent.....,"['gorgeous', 'gorgeous', 'blend', '..', 'love'...","['gorgeous', 'gorgeous', 'blend', '', 'love', ...","['gorgeous', 'gorgeous', 'blend', 'love', 'the...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce..."


In [10]:
label = label.drop("Unnamed: 0", axis =1)
label.head()

,name,label
0,Alien Mugler for women,2.0
1,Coco Mademoiselle Chanel for women,2.0
2,Black Orchid Tom Ford for women,2.0
3,Black Opium Yves Saint Laurent for women,2.0
4,Hypnotic Poison Christian Dior for women,2.0


In [11]:
#label추가하기
perfume = pd.merge(data,label, on = "name")

In [12]:
perfume.head()

,gender,name,accords,review,tokenized,only_english,longer_than_2_A,stopwords_removed,lemmatizated,label
0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'sample', 'this', 'today', 'and', 'yea...","['got', 'sample', 'today', 'year', 'old', 'dau...","['get', 'sample', 'today', 'year', 'old', 'dau...",2.0
1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\r\nI have ...,"['first', 'impressions.', 'test', 'on', 'blott...","['first', 'impressions', 'test', 'on', 'blotte...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impression', 'test', 'blotter', 'he...",2.0
2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...,"['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'best', 'friend...","['perfume', 'reminds', 'best', 'friend', 'actu...","['perfume', 'reminds', 'best', 'friend', 'actu...",2.0
3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...,"['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'feet', 'falling', 'fa...","['imagine', 'trip', 'foot', 'fall', 'face', 'f...",2.0
4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\r\nLove the scent.....,"['gorgeous', 'gorgeous', 'blend', '..', 'love'...","['gorgeous', 'gorgeous', 'blend', '', 'love', ...","['gorgeous', 'gorgeous', 'blend', 'love', 'the...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...",2.0


In [13]:
print(perfume.isnull().sum())
perfume = perfume.dropna()
print(perfume.isnull().sum())

gender               0
name                 0
accords              0
review               1
tokenized            0
only_english         0
longer_than_2_A      0
stopwords_removed    0
lemmatizated         0
label                0
dtype: int64
gender               0
name                 0
accords              0
review               0
tokenized            0
only_english         0
longer_than_2_A      0
stopwords_removed    0
lemmatizated         0
label                0
dtype: int64


In [14]:
def show_word_count_stat(data):
    num_word = []
    num_unique_words = []
    for item in data:
        num_word.append(len(str(item).split()))
        num_unique_words.append(len(set(str(item).split())))
 
    # 일반
    perfume['num_words'] = pd.Series(num_word)
    # 중복 제거
    perfume['num_unique_words'] = pd.Series(num_unique_words)
 
    x = data[0]
    x = str(x).split()
    print(len(x))
    print('perfume review 단어 평균 값 : ', perfume['num_words'].mean())
    print('perfume review 단어 중간 값', perfume['num_words'].median())
 
    print('perfume review 고유 단어 평균 값 : ', perfume['num_unique_words'].mean())
    print('perfume review 고유 단어 중간 값', perfume['num_unique_words'].median())
 
show_word_count_stat(perfume.lemmatizated)

76
perfume review 단어 평균 값 :  45.16187600590575
perfume review 단어 중간 값 33.0
perfume review 고유 단어 평균 값 :  38.351977228289066
perfume review 고유 단어 중간 값 30.0


## 훈련데이터 준비

In [15]:
#lemmatizated 데이터가 string형태로 들어가있음. 
#re.compile을 이용해서 영어만 뽑아 리스트형태로 저장
import re
def only_english(text):
  english = re.compile('[A-Za-z]+').findall(text)
  return english
perfume["lemmatizated_2"] = perfume["lemmatizated"].apply(only_english)
perfume["lemmatizated_str"] = perfume["lemmatizated_2"].str.join(' ')

In [16]:
X = list(perfume["lemmatizated_str"])
y = list(perfume["label"].astype(int))

In [17]:
#위 코드와의 차이점 :
from sklearn.model_selection import train_test_split
X = np.asarray(X)
y = np.asarray(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.5)

In [18]:
X_train, y_train, len(X_train),len(X_test)

(array(['feel like linterdit way try achieve execute well tuberose still strong linear like way think patchouli help make less sickly gummy sweet combine ambroxan make earthy complex bergamot first spray pear lingers add fresh watery lightness surface could smell come arm move around spray impressed silage unfortunately huge fan tuberoseheavy scent wouldnt purchase bottle think nice',
        'pinnacle gourmand fragrance scream delicious invite cozy love pineapple hazelnut nut cream straight smell like nutella begin sweet juicy pineapple note also begin smell vanilla blend chocolate fragrance dry get smooth patchouli base add spice surprise mugler fan usually patchouli main accord mugler fragrance bad thing actually love patchouli personally version blend smoothly background note allow experience note blend together make interesting invite enjoy nose enjoy even challenge fragrance line bother personally challenge many strong patchouli base many gravitate perfume patchouli note tone bac

In [19]:
max_len = max(len(l) for l in X)
max_len

7374

# 1. embedding : ELMo
- computes contextualized word representations using character-based word representations and biLSTM
- input : raw text stings or tokenized text strings
- 새로운 모델에 대해서 사용할때 예제를 통해서 자료구조가 어떤지 꼭 확인하기

👉딥러닝 입문 위키독스 보고 따라한것 -> (elmo/1 -> elmo/2)

🙅🏻‍♀️error : Signature expects multiple inputs. Use a dict. 

->해결 : signature: default (string), tokens으로 tokens에서 default형태로 바꿔서 사용



👉[최종] **블로그 참고....**

https://ichi.pro/ko/elmolo-ganglyeoghan-munmaeg-dan-eo-imbeding-eul-guchughaneun-bangbeob-al-abogi-40447472500805

https://wikidocs.net/33930

input 데이터를 elmo값으로 변환하여 모델 fit할때 elmo값으로 바뀐 데이터를 입력

In [20]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [21]:
list_train = [X_train[i:i+100] for i in range(0,X_train.shape[0],100)] #배치사이즈 :100개

In [ ]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x) for x in list_train]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  y_train,  
                                                  random_state=42, 
                                                  test_size=0.2)

In [ ]:
model = Sequential()
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
from sklearn.metrics import f1_score 
preds_valid = history.predict(xvalid)
f1_score(yvalid, preds_valid)

👇tensorflow hub tutorial보고 따라한 것

토큰화를 그대로 input으로 사용하려고 했음 -> 문제는 어떻게 활용할지 모르겠다는거... 



https://tfhub.dev/google/elmo/3

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

def embed_elmo(x):
  embeddings = elmo(x,
      signature="default", #token(batchisize와 max_length) or default(string으로 batchsize만)
      as_dict=True)["elmo"]
  return embeddings
#embeddings.shape #[훈련샘플수, 입력문자열 목록에서 가장 긴 문자열의 최대길이, ELMo벡터의 길이]

In [ ]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype="string")
embedding_layer =  Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='softmax')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', #멀티클래스 손실함수
              metrics=['accuracy'])

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))